<a href="https://colab.research.google.com/github/mhered/robotica_patito_pycon2025/blob/main/lerobot/training-act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-0.editable-py3-none-any.whl size=15308 sha256=a835316b156dd681b1eae7020bc9e1cbe439ad1f7b24003329c1b45ddecbe089
  Stored in directory: /tmp/pip-ephem-wheel-cache-f1lda20h/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: spam-mhered (spam-mhered-duckietown) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=mhered/recording-test`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!cd lerobot && lerobot-train \
  --dataset.repo_id=mhered/recording-test \
  --policy.type=act \
  --output_dir=outputs/train/act_so100_pato_test1 \
  --job_name=act_so100_pato_test \
  --policy.device=cuda \
  --wandb.enable=true \
  --policy.repo_id=mhered/my_act

INFO 2025-10-03 17:42:56 ot_train.py:148 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/il_sim_test0 \
  /content/lerobot/outputs/train/il_sim_test0/checkpoints/last/pretrained_model